# Data Wrangling for GEOG 458 Final Project
### Jin Ning Huang

## Setup (Import and Load the Data)

In [2]:
# import libraries
import pandas as pd
import numpy as np
import datetime
import csv
import calendar

In [3]:
# load the data and check what is inside
collision_data = pd.read_csv('Collisions.csv')
collision_data.head()

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,1.270739e+06,260119.782014,1,336736,338236,EA91716,Unmatched,Block,NaN,N 103RD ST BETWEEN WALLINGFORD AVE N AND MERID...,...,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,N
1,1.268027e+06,238245.867368,2,334709,336209,EA68406,Matched,Block,NaN,WESTLAKE AVE N BETWEEN HALLADAY ST AND NEWELL ST,...,Wet,Daylight,NaN,NaN,NaN,30,From opposite direction - all others,0,0,N
2,1.261730e+06,192489.632377,3,334261,335761,EA58089,Matched,Intersection,34902.0,26TH AVE SW AND SW ROXBURY ST,...,Dry,Daylight,NaN,NaN,NaN,11,From same direction - both going straight - bo...,0,0,N
3,1.264062e+06,246311.148464,4,334072,335572,EA61214,Unmatched,Intersection,25346.0,3RD AVE NW AND NW 50TH ST,...,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,Y
4,1.262526e+06,194078.844091,5,335976,337476,3876610,Matched,Block,NaN,SW BARTON PL BETWEEN 22ND AVE SW AND SW BARTON ST,...,Wet,Daylight,NaN,NaN,NaN,32,One parked--one moving,29573,0,Y


In [4]:
# check amount of incidents this data contains
collision_data.shape

(224539, 40)

## Formatting the Data

I changed the column "INCATE" into the datetime data type in order for me to grab certain values. I added three new columns: year, month, and tick. I get the individual row a month and a year in order to make the aggregation and filtering easier. I used the tick as a count that occured on that certain month. 

In [5]:
# change to datatime data type
collision_data['INCDATE'] =  pd.to_datetime(collision_data['INCDATE'], format='%Y-%m-%d')

In [6]:
# add year column
collision_data['year'] = collision_data['INCDATE'].dt.year

In [8]:
# add month column
collision_data['month'] = collision_data['INCDATE'].dt.month

In [9]:
# add month counts column
collision_data['tick'] = 1

In [10]:
# check at the left of the dataset
collision_data.head()

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,...,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,year,month,tick
0,1.270739e+06,260119.782014,1,336736,338236,EA91716,Unmatched,Block,NaN,N 103RD ST BETWEEN WALLINGFORD AVE N AND MERID...,...,NaN,NaN,,NaN,0,0,N,2020,12,1
1,1.268027e+06,238245.867368,2,334709,336209,EA68406,Matched,Block,NaN,WESTLAKE AVE N BETWEEN HALLADAY ST AND NEWELL ST,...,NaN,NaN,30,From opposite direction - all others,0,0,N,2020,9,1
2,1.261730e+06,192489.632377,3,334261,335761,EA58089,Matched,Intersection,34902.0,26TH AVE SW AND SW ROXBURY ST,...,NaN,NaN,11,From same direction - both going straight - bo...,0,0,N,2020,8,1
3,1.264062e+06,246311.148464,4,334072,335572,EA61214,Unmatched,Intersection,25346.0,3RD AVE NW AND NW 50TH ST,...,NaN,NaN,,NaN,0,0,Y,2020,9,1
4,1.262526e+06,194078.844091,5,335976,337476,3876610,Matched,Block,NaN,SW BARTON PL BETWEEN 22ND AVE SW AND SW BARTON ST,...,NaN,NaN,32,One parked--one moving,29573,0,Y,2020,11,1


## Filtering, Aggregating, then Printing a CSV

I decided to filter out four times: each for 2018, 2019, 2020, and 2018-2020 with the help of filtering. For each small dataset, I aggregating in a group-by way. Then, let this machines print out a new csv for the web plots.

#### For 2018 - 2020

In [11]:
# filter to 2018 - 2020
year_2018_to_2020 = collision_data[(collision_data['year'] >= 2018) & (collision_data['year'] <= 2020)]

In [12]:
# checking how many incidents on this time frame
year_2018_to_2020.shape

(30586, 43)

In [13]:
# checking each year's incidents
year_2018_to_2020.year.value_counts()

2018    12199
2019    11209
2020     7178
Name: year, dtype: int64

In [14]:
# aggregating by each months
year_all = year_2018_to_2020.groupby('month').sum()
# renaming to month names
dd=dict((enumerate(calendar.month_abbr)))
year_all = year_all.rename(index=dd,level=0)
# slice to only months and its counts
year_all = year_all['tick']
year_all

month
Jan    2957
Feb    2515
Mar    2417
Apr    2253
May    2522
Jun    2567
Jul    2621
Aug    2487
Sep    2414
Oct    2812
Nov    2503
Dec    2518
Name: tick, dtype: int64

In [15]:
# print out csv
year_all.to_csv(r'collision_2018_2020.csv')

#### For 2020

Same thing as previous dataset

In [16]:
year_2020 = collision_data[collision_data['year'] == 2020]

In [17]:
year_2020.shape

(7178, 43)

In [18]:
year_2020 = year_2020.groupby('month').sum()

In [19]:
dd=dict((enumerate(calendar.month_abbr)))
year_2020 = year_2020.rename(index=dd,level=0)

In [20]:
year_2020 = year_2020['tick']
year_2020

month
Jan    902
Feb    844
Mar    557
Apr    399
May    452
Jun    542
Jul    556
Aug    581
Sep    548
Oct    651
Nov    543
Dec    603
Name: tick, dtype: int64

In [21]:
year_2020.to_csv(r'collision_2020.csv')

#### For 2018

In [24]:
year_2018 = collision_data[collision_data['year'] == 2018]
year_2018 = year_2018.groupby('month').sum()
dd=dict((enumerate(calendar.month_abbr)))
year_2018 = year_2018.rename(index=dd,level=0)
year_2018 = year_2018['tick']
year_2018

month
Jan    1125
Feb     912
Mar     998
Apr     919
May    1050
Jun    1008
Jul    1064
Aug     987
Sep     957
Oct    1091
Nov    1026
Dec    1062
Name: tick, dtype: int64

In [25]:
year_2018.to_csv(r'collision_2018.csv')

#### For 2019

In [26]:
year_2019 = collision_data[collision_data['year'] == 2019]
year_2019 = year_2019.groupby('month').sum()
dd=dict((enumerate(calendar.month_abbr)))
year_2019 = year_2019.rename(index=dd,level=0)
year_2019 = year_2019['tick']
year_2019

month
Jan     930
Feb     759
Mar     862
Apr     935
May    1020
Jun    1017
Jul    1001
Aug     919
Sep     909
Oct    1070
Nov     934
Dec     853
Name: tick, dtype: int64

In [27]:
year_2019.to_csv(r'collision_2019.csv')